# Chapter 3. Transformer Anatomy

In [ ]:
from utils import *

setup_chapter()

In [ ]:
from math import sqrt

import torch
import torch.nn.functional as F
from torch import nn
from transformers import AutoConfig, AutoModel, AutoTokenizer

## Visualization using BERTViz

In [ ]:
from bertviz.neuron_view import show
from bertviz.transformers_neuron_view import BertModel

In [ ]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)

In [ ]:
text = "time flies like an arrow"
tokens = tokenizer.tokenize(text)
tokens

In [ ]:
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

Supermarket analogy when shopping for a dinner recipe to understand query, key, value:

- ingredient - query
- labels of grocereries in shelves - keys
- items on shelves - value
- BUT: smooth ingredients might lead to buy 1L milk, joghurt and cheese when looking for 2L of milk

## Implementing scaled dot-product attention

![](https://github.com/JungeAlexander/notebooks/raw/3832388ce6b0ff7868f2b4dc15c6105a29c21fc3//images/chapter03_attention-ops.png)

In [ ]:
model_ckpt = "bert-base-uncased"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
tokenizer.tokenize(text, add_special_tokens=True)

In [ ]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

In [ ]:
config = AutoConfig.from_pretrained(model_ckpt)

In [ ]:
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)

In [ ]:
token_emb

In [ ]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

In [ ]:
inputs_embeds[0, :, :3]

In [ ]:
inputs_embeds.type()

In [ ]:
query = inputs_embeds
key = inputs_embeds
value = inputs_embeds

dim_k = key.size(-1)

scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
scores.size()

In [ ]:
key.transpose(1, 2).shape

In [ ]:
weights = F.softmax(scores, dim=-1)
weights.size()

In [ ]:
weights.sum(dim=-1)

In [ ]:
def scaled_dot_product_attention(query, key, value):
    dim_k = key.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

## Implementing multi-headed attention

![](https://github.com/JungeAlexander/notebooks/raw/3832388ce6b0ff7868f2b4dc15c6105a29c21fc3//images/chapter03_multihead-attention.png)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        return scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
        )

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

In [ ]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

In [ ]:
from bertviz import head_view

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"

sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors="pt")
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

## The Feed-Forward Layer (position-wise)

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_output)
ff_outputs.size()

## Adding Layer Normalization

![](https://github.com/JungeAlexander/notebooks/raw/cf30f13ae7bf88b0b241bd89b29e385707c6e38e//images/chapter03_layer-norm.png)

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        x = x + self.attention(self.layer_norm_1(x))
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
encoder_layer(inputs_embeds).size()

## Positional Embeddings